In [1]:
"""
Implements a simple n-gram language model in PyTorch.
Acts as the correctness reference for all the other versions.
"""
import math
import torch
import torch.nn as nn
from torch.nn import functional as F
import optuna


C:\Users\50196\.conda\envs\deeplearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from common import RNG

## Hyperparameter 

In [3]:
context_length = 3 # if 3 tokens predict the 4th, this is a 4-gram model
embedding_size = 64
hidden_size = 512

learning_rate = 1e-3
batch_size = 64
num_steps = 50000

## MLP

In [4]:
class MLP(nn.Module):
    def __init__(self, vocab_size, context_length, embedding_size, hidden_size):
        super().__init__()
        self.wte = nn.Embedding(vocab_size, embedding_size)
        self.mlp = nn.Sequential(
            nn.Linear(context_length * embedding_size, hidden_size),
            nn.Tanh(),
            nn.GELU(),
            nn.Linear(hidden_size, vocab_size)
        )

    def forward(self, idx, targets=None):
        B, T = idx.size()
        emb = self.wte(idx)
        emb = emb.view(B, -1)
        logits = self.mlp(emb)
        loss = None
        if targets is not None:
            loss = F.cross_entropy(logits, targets)
        return logits, loss

In [5]:
def dataloader(tokens, context_length, batch_size, device):
    n = len(tokens)
    inputs, targets = [], []
    pos = 0
    while True:
        window = tokens[pos:pos + context_length + 1]
        inputs.append(window[:-1])
        targets.append(window[-1])
        if len(inputs) == batch_size:
            yield (torch.tensor(inputs, device=device), torch.tensor(targets, device=device))
            inputs, targets = [], []
        pos += 1
        if pos + context_length >= n:
            pos = 0

In [6]:
def eval_split(model, tokens, context_length, batch_size, device, max_batches=None):
    model.eval()
    total_loss = 0
    num_batches = len(tokens) // batch_size
    if max_batches is not None:
        num_batches = min(num_batches, max_batches)
    data_iter = dataloader(tokens, context_length, batch_size, device)
    for _ in range(num_batches):
        inputs, targets = next(data_iter)
        with torch.no_grad():
            logits, loss = model(inputs, targets)
        total_loss += loss.item()
    mean_loss = total_loss / num_batches
    return mean_loss

## Training

In [7]:
def train(model, train_tokens, val_tokens, context_length, batch_size, num_steps, learning_rate, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)
    train_data_iter = dataloader(train_tokens, context_length, batch_size, device)
    for step in range(num_steps):
        lr = learning_rate * 0.5 * (1 + math.cos(math.pi * step / num_steps))
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
        if step % 200 == 0 or step == num_steps - 1:
            train_loss = eval_split(model, train_tokens, context_length, batch_size, device, max_batches=20)
            val_loss = eval_split(model, val_tokens, context_length, batch_size, device)
            print(f'step {step} | train_loss {train_loss:.4f} | val_loss {val_loss:.4f} | lr {lr:e}')
        model.train()
        inputs, targets = next(train_data_iter)
        logits, loss = model(inputs, targets)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    return val_loss

In [8]:
random = RNG(1337)
# TODO: actually use this rng for the model initialization


## (optional) Optimize Hyperparameter

In [9]:
def objective(trial):
    # Define the hyperparameter search space
    embedding_size = trial.suggest_int('embedding_size', 16, 64)
    hidden_size = trial.suggest_int('hidden_size', 256, 1024)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128])
    context_length = trial.suggest_int('context_length', 2, 5)

    # Load data and preprocess
    train_text = open('data/train.txt', 'r').read()
    uchars = sorted(list(set(train_text)))
    vocab_size = len(uchars)
    char_to_token = {c: i for i, c in enumerate(uchars)}
    train_tokens = [char_to_token[c] for c in train_text]
    val_tokens = [char_to_token[c] for c in open('data/val.txt', 'r').read()]

    # Create the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = MLP(vocab_size, context_length, embedding_size, hidden_size).to(device)

    # Train the model
    num_steps = 50000
    val_loss = train(model, train_tokens, val_tokens, context_length, batch_size, num_steps, learning_rate, device)
    
    return val_loss

In [10]:
if __name__ == "__main__":
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=50)
    
    print('Best trial:')
    trial = study.best_trial

    print('  Value: {}'.format(trial.value))

    print('  Params: ')
    for key, value in trial.params.items():
        print('    {}: {}'.format(key, value))

[I 2024-07-19 22:00:11,261] A new study created in memory with name: no-name-f27561d9-03d1-4777-a7c7-83960f9f0297
C:\Users\50196\AppData\Local\Temp\ipykernel_46372\779706708.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


step 0 | train_loss 3.3023 | val_loss 3.3001 | lr 3.140955e-04
step 200 | train_loss 2.3936 | val_loss 2.4220 | lr 3.140831e-04
step 400 | train_loss 2.3212 | val_loss 2.3475 | lr 3.140459e-04
step 600 | train_loss 2.2930 | val_loss 2.3173 | lr 3.139839e-04
step 800 | train_loss 2.2743 | val_loss 2.2917 | lr 3.138971e-04
step 1000 | train_loss 2.2544 | val_loss 2.2694 | lr 3.137856e-04
step 1200 | train_loss 2.2414 | val_loss 2.2576 | lr 3.136493e-04
step 1400 | train_loss 2.2335 | val_loss 2.2465 | lr 3.134883e-04
step 1600 | train_loss 2.2218 | val_loss 2.2352 | lr 3.133026e-04
step 1800 | train_loss 2.1903 | val_loss 2.2264 | lr 3.130922e-04
step 2000 | train_loss 2.1905 | val_loss 2.2186 | lr 3.128571e-04
step 2200 | train_loss 2.1830 | val_loss 2.2111 | lr 3.125975e-04
step 2400 | train_loss 2.1769 | val_loss 2.2046 | lr 3.123133e-04
step 2600 | train_loss 2.1821 | val_loss 2.1989 | lr 3.120046e-04
step 2800 | train_loss 2.1704 | val_loss 2.1931 | lr 3.116714e-04
step 3000 | train

[I 2024-07-19 22:01:11,896] Trial 0 finished with value: 2.056985463414873 and parameters: {'embedding_size': 47, 'hidden_size': 738, 'learning_rate': 0.0003140954960818081, 'batch_size': 128, 'context_length': 3}. Best is trial 0 with value: 2.056985463414873.


step 49999 | train_loss 1.9644 | val_loss 2.0570 | lr 3.099998e-13
step 0 | train_loss 3.3026 | val_loss 3.3027 | lr 1.288762e-04


C:\Users\50196\AppData\Local\Temp\ipykernel_46372\779706708.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)


...

[I 2024-07-19 22:22:46,558] Trial 19 finished with value: 2.0004928798547814 and parameters: {'embedding_size': 21, 'hidden_size': 521, 'learning_rate': 0.0006711249701238267, 'batch_size': 32, 'context_length': 5}. Best is trial 16 with value: 1.9904905743896961.


step 49999 | train_loss 1.8998 | val_loss 2.0005 | lr 6.623738e-13
step 0 | train_loss 3.3142 | val_loss 3.3156 | lr 8.683602e-04
step 200 | train_loss 2.3656 | val_loss 2.4419 | lr 8.683259e-04
step 400 | train_loss 2.3226 | val_loss 2.3692 | lr 8.682231e-04
step 600 | train_loss 2.3220 | val_loss 2.3399 | lr 8.680517e-04
step 800 | train_loss 2.2796 | val_loss 2.2960 | lr 8.678118e-04
step 1000 | train_loss 2.2567 | val_loss 2.2754 | lr 8.675035e-04
step 1200 | train_loss 2.2439 | val_loss 2.2696 | lr 8.671267e-04
step 1400 | train_loss 2.2744 | val_loss 2.2659 | lr 8.666815e-04
step 1600 | train_loss 2.2593 | val_loss 2.2411 | lr 8.661680e-04
step 1800 | train_loss 2.2540 | val_loss 2.2411 | lr 8.655864e-04
step 2000 | train_loss 2.2448 | val_loss 2.2326 | lr 8.649366e-04
step 2200 | train_loss 2.2236 | val_loss 2.2207 | lr 8.642187e-04
step 2400 | train_loss 2.1911 | val_loss 2.2185 | lr 8.634330e-04
step 2600 | train_loss 2.2097 | val_loss 2.2007 | lr 8.625795e-04
step 2800 | trai

[W 2024-07-19 22:23:21,219] Trial 20 failed with parameters: {'embedding_size': 32, 'hidden_size': 654, 'learning_rate': 0.0008683602070624887, 'batch_size': 32, 'context_length': 4} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\50196\.conda\envs\deeplearning\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\50196\AppData\Local\Temp\ipykernel_46372\779706708.py", line 23, in objective
    val_loss = train(model, train_tokens, val_tokens, context_length, batch_size, num_steps, learning_rate, device)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\50196\AppData\Local\Temp\ipykernel_46372\2298479787.py", line 15, in train
    loss.backward()
  File "C:\Users\50196\.conda\envs\deeplearning\Lib\site-packages\torch\_tensor.py", line 525, in backwa

step 25200 | train_loss 1.9767 | val_loss 2.0272 | lr 4.287242e-04


KeyboardInterrupt: 